In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.compose import ColumnTransformer

import matplotlib.pyplot as plt #visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("Set2")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

from google.colab import drive
drive.mount('/content/drive')

# set paths-- change to YOUR path
DATA_FOLDER = '/content/drive/MyDrive/Colab Notebooks/data/'
FIGURE_FOLDER = '/content/drive/MyDrive/Colab Notebooks/img/'
RESULT_FOLDER = '/content/drive/MyDrive/Colab Notebooks/results/'

# Create directories if they don't exist
import os
os.makedirs(FIGURE_FOLDER, exist_ok=True)
os.makedirs(RESULT_FOLDER, exist_ok=True)

def check_for_nulls(df):
    """Print out null counts for a dataframe."""
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        print("Null counts:")
        print(null_counts[null_counts > 0])
    else:
        print("No nulls found.")

def load_data(mode='TRAIN', sample_size=None):
    """
    Load and merge all relevant data files.

    Parameters:
    mode (str): 'TRAIN' or 'TEST'
    sample_size (int): If provided, sample the data to speed up processing

    Returns:
    DataFrame with all features combined
    """
    # Load categorical metadata - handle different naming conventions between TRAIN and TEST
    if mode == 'TRAIN':
        cat_meta = pd.read_excel(f"{DATA_FOLDER}{mode}/{mode}_CATEGORICAL_METADATA.xlsx")
    else:  # TEST mode
        cat_meta = pd.read_excel(f"{DATA_FOLDER}{mode}/{mode}_CATEGORICAL.xlsx")

    # Load quantitative metadata
    quant_meta = pd.read_excel(f"{DATA_FOLDER}{mode}/{mode}_QUANTITATIVE_METADATA.xlsx")

    # Load functional connectome matrices (fix spelling)
    fcm = pd.read_csv(f"{DATA_FOLDER}{mode}/{mode}_FUNCTIONAL_CONNECTOME_MATRICES.csv")

    # Optional sampling for faster development/testing
    if sample_size and mode == 'TRAIN':
        cat_meta = cat_meta.sample(sample_size, random_state=42)
        participant_ids = cat_meta['participant_id'].values
        quant_meta = quant_meta[quant_meta['participant_id'].isin(participant_ids)]
        fcm = fcm[fcm['participant_id'].isin(participant_ids)]

    # Merge all data sources
    data = cat_meta.merge(quant_meta, on='participant_id', how='left')
    data = data.merge(fcm, on='participant_id', how='left')

    return data

# Section 1--Data Loading and Exploration =====
print("Loading data...")

# Use a smaller sample size for faster development
SAMPLE_SIZE = 300  # Set to None to use full dataset

# Load training data
train = load_data(mode='TRAIN', sample_size=SAMPLE_SIZE)
y = pd.read_excel(f"{DATA_FOLDER}TRAIN/TRAINING_SOLUTIONS.xlsx")

# If we sampled the train data, filter y accordingly
if SAMPLE_SIZE:
    y = y[y['participant_id'].isin(train['participant_id'])]

# Load test data - we can use a small sample for testing, then run on full dataset for final submission
test = load_data(mode='TEST', sample_size=100)  # Sample for development
print(f"Train: {train.shape}, Test: {test.shape}")

# Load sample submission
sub = pd.read_excel(f"{DATA_FOLDER}SAMPLE_SUBMISSION.xlsx")

# Set participant_id as index for easier handling
train.set_index('participant_id', inplace=True)
test.set_index('participant_id', inplace=True)
y_train = y.set_index('participant_id')

# Define targets and features
targets = ['ADHD_Outcome', 'Sex_F']
features = test.columns

# Check for nulls in the data
print("Checking for nulls in training data:")
check_for_nulls(train)
print("\nChecking for nulls in test data:")
check_for_nulls(test)

#Section 2-- Exploratory Data Analysis
print("\nPerforming exploratory data analysis...")

# Plot target distributions
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
for col, ax in zip(y_train.columns, axs):
    counts = y_train[col].value_counts()
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'{col} Distribution')
plt.tight_layout()
plt.savefig(f"{FIGURE_FOLDER}target_distributions.png", dpi=300)
plt.close()

# Identify features for log transformation
log_features = [f for f in features if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0.5]
print(f"Number of features to be log-transformed: {len(log_features)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data...
Train: (300, 19928), Test: (304, 19928)
Checking for nulls in training data:
Null counts:
PreInt_Demos_Fam_Child_Ethnicity     5
MRI_Track_Age_at_Scan               97
dtype: int64

Checking for nulls in test data:
Null counts:
PreInt_Demos_Fam_Child_Ethnicity     3
PreInt_Demos_Fam_Child_Race          6
Barratt_Barratt_P1_Edu               1
Barratt_Barratt_P1_Occ               1
Barratt_Barratt_P2_Edu              36
Barratt_Barratt_P2_Occ              42
EHQ_EHQ_Total                        1
ColorVision_CV_Score                 9
APQ_P_APQ_P_CP                      15
APQ_P_APQ_P_ID                      15
APQ_P_APQ_P_INV                     15
APQ_P_APQ_P_OPD                     15
APQ_P_APQ_P_PM                      15
APQ_P_APQ_P_PP                      15
SDQ_SDQ_Conduct_Problems            30
SDQ_SDQ_Difficulties_Total          30
SDQ

In [ ]:
# Section 2--Exploratory Data Analysis
print("\nEDA continued...")

# Plot target distributions
fig, axs = plt.subplots(1, 2, figsize=(10, 5))
for col, ax in zip(y_train.columns, axs):
    counts = y_train[col].value_counts()
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'{col} Distribution')
plt.tight_layout()
plt.savefig(f"{FIGURE_FOLDER}target_distributions.png", dpi=300)
plt.close()

# Identify features for log transformation
log_features = [f for f in features if (train[f] >= 0).all() and scipy.stats.skew(train[f]) > 0.5]
print(f"Number of features to be log-transformed: {len(log_features)}")

# Look at distribution of numerical features with highest skew (optional visualization)
high_skew_features = sorted([(f, scipy.stats.skew(train[f])) for f in features
                            if (train[f] >= 0).all() and not pd.isna(scipy.stats.skew(train[f]))],
                            key=lambda x: abs(x[1]), reverse=True)[:5]
print("\nTop 5 most skewed features:")
for feat, skew_val in high_skew_features:
    print(f"{feat}: {skew_val:.4f}")

# Section 3 -- Feature Engineering
print("\nPerforming feature engineering...")

# Handle missing values strategically before further processing
print("Handling missing values...")

# Identify categorical and numerical columns based on data types
categorical_cols = train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# For numerical columns with nulls, impute with median (more robust to outliers than mean)
num_imputer = SimpleImputer(strategy='median')
train_num = pd.DataFrame(
    num_imputer.fit_transform(train[numerical_cols]),
    columns=numerical_cols,
    index=train.index
)

# For categorical columns with nulls, impute with most frequent value
cat_imputer = SimpleImputer(strategy='most_frequent')
if categorical_cols:
    train_cat = pd.DataFrame(
        cat_imputer.fit_transform(train[categorical_cols]),
        columns=categorical_cols,
        index=train.index
    )
    # Combine imputed data back
    train_imputed = pd.concat([train_num, train_cat], axis=1)
else:
    train_imputed = train_num

# Apply the same imputation to test data
test_num = pd.DataFrame(
    num_imputer.transform(test[numerical_cols]),
    columns=numerical_cols,
    index=test.index
)
if categorical_cols:
    test_cat = pd.DataFrame(
        cat_imputer.transform(test[categorical_cols]),
        columns=categorical_cols,
        index=test.index
    )
    test_imputed = pd.concat([test_num, test_cat], axis=1)
else:
    test_imputed = test_num

# Function to create interaction features for given columns
def create_interaction_features(df, cols):
    """Create interaction terms between specified columns."""
    df_copy = df.copy()

    # Create meaningful pairwise interactions
    for i, col1 in enumerate(cols):
        for col2 in cols[i+1:]:
            df_copy[f"{col1}_{col2}_mult"] = df[col1] * df[col2]

    return df_copy

# Create features for SDQ and APQ columns specifically - these are known to be important for ADHD prediction
# from clinical research (Strengths and Difficulties Questionnaire & Alabama Parenting Questionnaire)
sdq_cols = [col for col in train_imputed.columns if 'SDQ_' in col]
apq_cols = [col for col in train_imputed.columns if 'APQ_' in col]

print(f"Number of SDQ features: {len(sdq_cols)}")
print(f"Number of APQ features: {len(apq_cols)}")

# Identify most important features for each target using SelectKBest
def get_important_features(X, y, k=20):
    """Get the k most important features for target y."""
    selector = SelectKBest(f_classif, k=k)
    selector.fit(X, y)
    mask = selector.get_support()
    return list(X.columns[mask])

# Get important features for each target
adhd_features = get_important_features(train_imputed, y_train['ADHD_Outcome'], k=40)
sex_features = get_important_features(train_imputed, y_train['Sex_F'], k=40)

# Print some insights about the important features
print("\nTop 5 important features for ADHD prediction:")
for feature in adhd_features[:5]:
    print(f"- {feature}")

print("\nTop 5 important features for Sex prediction:")
for feature in sex_features[:5]:
    print(f"- {feature}")

# Combine unique important features from both targets
important_features = list(set(adhd_features + sex_features + sdq_cols + apq_cols))
print(f"\nNumber of selected important features: {len(important_features)}")

# Create interaction features for both train and test - limit to most important features to avoid explosion
top_features_for_interactions = adhd_features[:3] + sex_features[:3]
top_features_for_interactions = list(set(top_features_for_interactions))

X_train_int = create_interaction_features(train_imputed[important_features], top_features_for_interactions)
X_test_int = create_interaction_features(test_imputed[important_features], top_features_for_interactions)



EDA continued...
Number of features to be log-transformed: 18

Top 5 most skewed features:
ColorVision_CV_Score: -4.3239
APQ_P_APQ_P_INV: -2.6976
APQ_P_APQ_P_PP: -2.6966
Barratt_Barratt_P1_Edu: -1.9190
APQ_P_APQ_P_CP: 1.4402

Performing feature engineering...
Handling missing values...
Number of SDQ features: 9
Number of APQ features: 6

Top 5 important features for ADHD prediction:
- 2throw_98thcolumn
- 4throw_145thcolumn
- 5throw_113thcolumn
- 5throw_157thcolumn
- 7throw_77thcolumn

Top 5 important features for Sex prediction:
- 0throw_79thcolumn
- 2throw_198thcolumn
- 4throw_14thcolumn
- 4throw_112thcolumn
- 6throw_190thcolumn

Number of selected important features: 94


In [ ]:
# Section 4-- Model Building and Evaluation
print("\nBuilding and evaluating models...")

# Split data for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_int, y_train, test_size=0.25, random_state=42, stratify=y_train['ADHD_Outcome']
)

# Define different model configurations to test
models = {
    'RidgeClassifier': MultiOutputClassifier(RidgeClassifier(alpha=10, class_weight='balanced', random_state=42)),
    'LogisticRegression': MultiOutputClassifier(LogisticRegression(max_iter=1000, class_weight='balanced',
                                                                C=0.1, solver='liblinear', random_state=42)),
    'RandomForest': MultiOutputClassifier(RandomForestClassifier(n_estimators=100, max_depth=15,
                                                              class_weight='balanced', random_state=42)),
    'GradientBoosting': MultiOutputClassifier(GradientBoostingClassifier(n_estimators=100, learning_rate=0.05,
                                                                      max_depth=5, random_state=42))
}

# Define preprocessing pipeline
def create_pipeline(model, n_components=50):
    """Create a preprocessing and model pipeline."""
    # Since we've already handled missing values above, we can simplify this pipeline
    return make_pipeline(
        ColumnTransformer([
            ('log_transform', FunctionTransformer(np.log1p),
             [col for col in X_train_int.columns if col in log_features])
        ], remainder='passthrough'),
        StandardScaler(),
        PCA(n_components=n_components, random_state=42),
        model
    )

# Train and evaluate each model
results = {}
for name, model in models.items():
    print(f"\nTraining {name}...")

    # Create and fit pipeline
    pipeline = create_pipeline(model, n_components=min(50, X_train_split.shape[0] - 1))
    pipeline.fit(X_train_split, y_train_split)

    # Predict on validation set
    y_pred = pipeline.predict(X_val_split)

    # Calculate metrics
    f1_adhd = f1_score(y_val_split['ADHD_Outcome'], y_pred[:, 0])
    f1_sex = f1_score(y_val_split['Sex_F'], y_pred[:, 1])
    f1_avg = (f1_adhd + f1_sex) / 2

    # Store results
    results[name] = {
        'f1_adhd': f1_adhd,
        'f1_sex': f1_sex,
        'f1_avg': f1_avg,
        'pipeline': pipeline
    }

    print(f"{name} F1 Scores - ADHD: {f1_adhd:.4f}, Sex: {f1_sex:.4f}, Average: {f1_avg:.4f}")
    print(classification_report(y_val_split, y_pred, target_names=targets))

# Find best model
best_model_name = max(results.keys(), key=lambda k: results[k]['f1_avg'])
best_pipeline = results[best_model_name]['pipeline']
print(f"\nBest model: {best_model_name} with average F1 score: {results[best_model_name]['f1_avg']:.4f}")



Building and evaluating models...

Training RidgeClassifier...
RidgeClassifier F1 Scores - ADHD: 0.8991, Sex: 0.9206, Average: 0.9099
              precision    recall  f1-score   support

ADHD_Outcome       0.88      0.92      0.90        53
       Sex_F       0.94      0.91      0.92        32

   micro avg       0.90      0.92      0.91        85
   macro avg       0.91      0.92      0.91        85
weighted avg       0.90      0.92      0.91        85
 samples avg       0.81      0.81      0.80        85


Training LogisticRegression...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression F1 Scores - ADHD: 0.8972, Sex: 0.9206, Average: 0.9089
              precision    recall  f1-score   support

ADHD_Outcome       0.89      0.91      0.90        53
       Sex_F       0.94      0.91      0.92        32

   micro avg       0.91      0.91      0.91        85
   macro avg       0.91      0.91      0.91        85
weighted avg       0.91      0.91      0.91        85
 samples avg       0.81      0.79      0.79        85


Training RandomForest...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


RandomForest F1 Scores - ADHD: 0.8870, Sex: 0.4651, Average: 0.6760
              precision    recall  f1-score   support

ADHD_Outcome       0.82      0.96      0.89        53
       Sex_F       0.91      0.31      0.47        32

   micro avg       0.84      0.72      0.77        85
   macro avg       0.87      0.64      0.68        85
weighted avg       0.86      0.72      0.73        85
 samples avg       0.75      0.65      0.68        85


Training GradientBoosting...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


GradientBoosting F1 Scores - ADHD: 0.9074, Sex: 0.7308, Average: 0.8191
              precision    recall  f1-score   support

ADHD_Outcome       0.89      0.92      0.91        53
       Sex_F       0.95      0.59      0.73        32

   micro avg       0.91      0.80      0.85        85
   macro avg       0.92      0.76      0.82        85
weighted avg       0.91      0.80      0.84        85
 samples avg       0.77      0.71      0.73        85


Best model: RidgeClassifier with average F1 score: 0.9099


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Section 5--Final Model and Submission
print("\nTraining final model and generating submission...")

# training best model on full training data
final_pipeline = create_pipeline(
    models[best_model_name],
    n_components=min(100, train.shape[0] - 1)
)
final_pipeline.fit(X_train_int, y_train)

# Predict on test data
y_pred = final_pipeline.predict(X_test_int)

# Save predictions
sub_df = pd.DataFrame({
    'participant_id': test.index,
    'ADHD_Outcome': y_pred[:, 0],
    'Sex_F': y_pred[:, 1]
})
sub_df.to_csv(f'{RESULT_FOLDER}submission.csv', index=False)
print(f"Submission saved to {RESULT_FOLDER}submission.csv")


Training final model and generating submission...
Submission saved to /content/drive/MyDrive/Colab Notebooks/results/submission.csv


In [ ]:
# Section 6-- Feature Importance Analysis
print("\nAnalyzing feature importance...")

if best_model_name in ['RandomForest', 'GradientBoosting']:
    # Get feature importance for tree-based models
    importances = np.zeros(X_train_int.shape[1])

    for i, estimator in enumerate(final_pipeline[-1].estimators_):
        if hasattr(estimator, 'feature_importances_'):
            importances += estimator.feature_importances_

    importances = importances / len(final_pipeline[-1].estimators_)

    # Plot top 20 features
    indices = np.argsort(importances)[::-1][:20]
    plt.figure(figsize=(12, 8))
    plt.title(f'Top 20 Feature Importances for {best_model_name}')
    plt.barh(range(20), importances[indices])
    plt.yticks(range(20), X_train_int.columns[indices])
    plt.tight_layout()
    plt.savefig(f"{FIGURE_FOLDER}feature_importance.png", dpi=300)
    plt.close()

print("Analysis complete!")



Analyzing feature importance...
Analysis complete!
